In [ ]:
import sys
sys.path.append('../')
%reload_ext autoreload 
%autoreload 2

In [ ]:
from general_functions import *
from cross_country_functions import *

In [ ]:
from nigeria_functions import prepare_target_roster_geo_nga, actual_predicted_quantiles_weights_nga, actual_predicted_urban_weights_nga

In [ ]:
from classification_class import *

In [ ]:
from visualisations import plot_precision_recall_curve, plot_roc_curve, visualise_actual_predicted_map, scatterplot_actual_predicted_percentage, barplots_sanity_wealth, barplots_sanity_area, create_roc_curve


### Prepare the data:

In [ ]:
features = pd.read_csv('../data/all_features.csv', index_col=0)
features.index = features.index.astype(str)

In [ ]:
path_to_targets = '../data/ethiopia_nigeria_targets.csv'

In [ ]:
path_to_inventory = '../data/inventory_small.xlsx'

In [ ]:
hh_roster_nga_weights = pd.read_csv('../data/nga/hh_roster_nga_weights.csv', index_col=0)
hh_roster_nga_weights.hhid = hh_roster_nga_weights.hhid.astype(str)
df_lsms_ram_roster_adm1 = pd.read_csv('../data/nga/df_lsms_ram_roster_adm1_nga.csv', index_col=0)
df_lsms_ram_roster_adm1.hhid = df_lsms_ram_roster_adm1.hhid.astype(str)

create list to save the performances for each threshold

In [ ]:
l_dict_thresholds_performances = []

Set threshold probability:

In [ ]:
#manually set the probability threshold, it can be [0.15, 0.85], None is 0.5
threshold_probability = 0.5 

In [ ]:
#True if you want to save the images
save_image = False

In [ ]:
#set the micronutrient
t = 'mimi_simple'

In [ ]:
all_target_names = {'zn_ai': 'zinc', 'va_ai': 'vitamin A', 'fol_ai': 'folate', 
                    'vb12_ai': 'vitamin B12', 'fe_ai':'iron', 'mimi_simple': 'overall micronutrient'}

In [ ]:
#for t in ['zn_ai', 'va_ai', 'fol_ai', 'vb12_ai', 'fe_ai', 'mimi_simple']:
for t in ['mimi_simple']:
    
    print(t)
    
    target_name = all_target_names[t]
    
    y, indexes = prepares_crosscountry_inputs('Ethiopia','Nigeria', t, path_to_targets= path_to_targets)
     #---Model
    #get the best random state
    best_random_state = pd.read_csv('../data/results/perf_%s_ETH_undersampling_3.2_xgboost.csv'%t, index_col=0).best_random_state[0]
    #set the classification object
    classification = Classification(y=y, data_all=features, cross_country=True, train_indexes=indexes,
                                    type_target=t, random_state=best_random_state, sampling='undersampling',
                                    sampling_strategy=1)
    #train the model using the best hyperparameters
    model = classification.xgbclassification_best_model('../data/results/besthyper_%s_ETH_undersampling_3.2_XGBoost.csv'%t)
    
    #probalilities
    y_proba = classification.y_proba(model)
    
    #---Performance---
    if threshold_probability != None:
        predictions = predictions_proba(y_proba, threshold_probability)
    else:
        predictions = classification.predictions(model)
    perf_dict = classification.perf_ind_classification(predictions)
    
    array_precision, array_recall, average_pre_recall = calculates_precision_recall_auc(y_proba, classification, drop_intermediate=True)
    #append average_pre_recall on perf_dict
    perf_dict['average_pre_recall'] = average_pre_recall
    array_fpr, array_tpr, rocauc_score = calculates_roc_auc(y_proba, classification, drop_intermediate=True)
    #append roc scores on perf_dict
    perf_dict['rocauc_score'] = rocauc_score
    #plot and save precision-recall curve and roc curve
    fig = plot_precision_recall_curve(classification, array_recall, array_precision, save=save_image, mn=t, iso3='NGA', title='risk of inadequate %s intake \n (Ethiopia to Nigeria)' %target_name)
    plot_roc_curve(classification, array_fpr, array_tpr, drop_intermediate=True, save=save_image, mn=t, iso3='NGA', title='risk of inadequate %s intake \n (Ethiopia to Nigeria)' %target_name)
    #calculate adjusted precision-recall and roc values and save them on the perf_dict
    adjusted_roc_auc, adjusted_average_pre_recall = get_adjusted_values(classification, rocauc_score, average_pre_recall)
    perf_dict['adjusted_rocauc'] = adjusted_roc_auc
    perf_dict['adjusted_average_pre_recall'] = adjusted_average_pre_recall

     #---Maps---
    geo_df = get_geodata(adm0=182)
    
    predicted_actual_target = classification.df_predicted_actual(predictions, t, save=False)
    target_roster = preprocess_roster_actual_predicted(predicted_actual_target, path_to_inventory, survey_id='NGA_2018_LSS_v01_M', hh_roster=hh_roster_nga_weights)
    target_roster_geo = prepare_target_roster_geo_nga(df_lsms_ram_roster_adm1, target_roster)
    
    actual_predicted_perc = calculate_actual_predicted(target_roster_geo, survey_id='NGA_2018_LSS_v01_M', path_to_inventory=path_to_inventory, weights=True, rank=True)
    for column in ['actual', 'predicted']:
        if column=='actual':
            title=''
        else:
            title='(T=%s)'%threshold_probability
        visualise_actual_predicted_map(geo_df, actual_predicted_perc, column, comparison=True, iso3='NGA', threshold = threshold_probability, save=save_image, title=title,  mn=target_name)
    for column2 in ['actual_rank', 'predicted_rank']:
        visualise_actual_predicted_map(geo_df, actual_predicted_perc, column2, comparison=False, iso3='NGA', threshold = threshold_probability, save=save_image, mn=target_name)

    #---Save performances---:
    for k, v in perf_dict.items():
        perf_dict[k] = [v]
    df_perf_dict = pd.DataFrame(perf_dict)
    
    #create a list and save performances for every threshold
    dict_thresholds_performances = {'threshold': threshold_probability, 'recall': df_perf_dict.recall[0], 'precision': df_perf_dict.precision[0], 'specificity': df_perf_dict.specificity[0]}
    l_dict_thresholds_performances.append(dict_thresholds_performances)

In [ ]:
scatterplot_actual_predicted_percentage(actual_predicted_perc, title='Percentage of risk of inadequate overall intake (ETH to NGA)', iso3='NGA', path = None, limin=0.1, limax=1, diff_threshold=0.95, fontsize=12, save=False)

## Sanity checks

In [ ]:
df_quantiles_weights = pd.read_csv('../data/nga/df_quantiles_weights_nga.csv', index_col=0) 

In [ ]:
df_quantiles_weights = actual_predicted_quantiles_weights_nga(df_quantiles_weights, predicted_actual_target)

In [ ]:
barplots_sanity_wealth(df_quantiles_weights, column='actual', country='nga', title='Actual percentage', title_x='Wealth quintile', save=False)


In [ ]:
barplots_sanity_wealth(df_quantiles_weights, column='predicted', T=threshold_probability, country='nga',  title='Predicted percentage (T=%s)'%threshold_probability, title_x='Wealth quintile', save=False)


In [ ]:
predicted_actual_urban_weights = actual_predicted_urban_weights_nga(predicted_actual_target, hh_roster_nga_weights)

In [ ]:
barplots_sanity_area(predicted_actual_urban_weights, column='actual', title='Actual percentage', title_x='Area', country='nga', save=False)

In [ ]:
barplots_sanity_area(predicted_actual_urban_weights, column='predicted', T=threshold_probability, title='Predicted percentage (T=%s)'%threshold_probability, title_x='Area', country='nga', save=False)

In [ ]:
#you can run this function after creating the dictionary with the postive/negative rates for each probability threshold
create_roc_curve(mn=t, classification=classification, array_fpr=array_fpr, array_tpr=array_tpr, title = "Ethiopia to Nigeria", countries='eth_to_nga', l_dict_thresholds_performances=l_dict_thresholds_performances, save=False)